# Exploring Correlations with Heatmaps and Scatterplots

In [1]:
import numpy as np
import numpy.polynomial.polynomial as poly      # linear regression
import pandas as pd

import pickle

import re

# opening external coordinates
import json

# opening urls
from urllib.request import urlopen

# benchmarking
from time import time

# date/time operations
from datetime import datetime, timedelta
from pytz import timezone

# plotting
import altair as alt
from altair import datum
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# for US map
from vega_datasets import data

# large datasets
alt.data_transformers.enable('data_server');

pd.options.display.max_rows = 250
pd.options.display.max_columns = 250

In [2]:
#hide
def optimize(df):
    '''
    Optimizes the data types in a pandas dataframe.
    '''
    dft = df.copy()
    # converts to datetime if possible
    dft = dft.apply(lambda col:
        pd.to_datetime(col, errors='ignore') if col.dtypes=='object' else col)
    
    # if there are less than half as many unique values as there are rows, convert to category
    for col in dft.select_dtypes(include='object'):
        if len(dft[col].unique()) / len(df[col]) < 0.5:
            dft[col] = dft[col].astype('category')
            
    # downcasts numeric columns if possible
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='integer') if col.dtypes=='int64' else col)
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='float') if col.dtypes=='float64' else col)
    
    return dft

In [20]:
with open('../data/processed/nyt_df.p', 'rb') as f:
    nyt_df = pickle.load(f)
nyt_df.head()

,date,county,state,fips,cases,deaths,cases_per_100k,deaths_per_100k,new_cases,new_deaths,new_cases_per_100k,new_deaths_per_100k,new_cases_15d,new_deaths_15d,new_cases_per_100k_15d,new_deaths_per_100k_15d,new_cases_15sg,new_deaths_15sg,new_cases_per_100k_15sg,new_deaths_per_100k_15sg,delta_new_cases,delta_new_deaths,delta_new_cases_per_100k,delta_new_deaths_per_100k,delta_new_cases_15d,delta_new_deaths_15d,delta_new_cases_per_100k_15d,delta_new_deaths_per_100k_15d,delta_new_cases_15sg,delta_new_deaths_15sg,delta_new_cases_per_100k_15sg,delta_new_deaths_per_100k_15sg,days,mortality_rate,mortality_rate_15d
0,2020-01-21,Snohomish,Washington,53061,1,0,0.121642,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0,0.0,NaN
1,2020-01-22,Snohomish,Washington,53061,1,0,0.121642,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,1,0.0,NaN
2,2020-01-23,Snohomish,Washington,53061,1,0,0.121642,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,2,0.0,NaN
3,2020-01-24,Cook,Illinois,17031,1,0,0.019417,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.091667,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.025,0.0,0.0,0.0,3,0.0,NaN
4,2020-01-24,Snohomish,Washington,53061,1,0,0.121642,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,3,0.0,NaN


In [4]:
with open('../data/processed/info_df.p', 'rb') as f:
    info_df = pickle.load(f)
info_df.head()

,state_fips,state,county,fips,tot_pop,tot_male,tot_female,tot_pop_white_male,tot_pop_white_female,tot_pop_black_male,tot_pop_black_female,tot_pop_native_male,tot_pop_native_female,tot_pop_asian_male,tot_pop_asian_female,tot_pop_pacific_male,tot_pop_pacific_female,tot_pop_twoplus_male,tot_pop_twoplus_female,tot_pop_hispanic_male,tot_pop_hispanic_female,tot_pop_white,tot_pop_black,tot_pop_native,tot_pop_asian,tot_pop_pacific,tot_pop_twoplus,tot_pop_hispanic,age_pop,age_male,age_female,age_pop_white_male,age_pop_white_female,age_pop_black_male,age_pop_black_female,age_pop_native_male,age_pop_native_female,age_pop_asian_male,age_pop_asian_female,age_pop_pacific_male,age_pop_pacific_female,age_pop_twoplus_male,age_pop_twoplus_female,age_pop_hispanic_male,age_pop_hispanic_female,age_pop_white,age_pop_black,age_pop_native,age_pop_asian,age_pop_pacific,age_pop_twoplus,age_pop_hispanic,area_land,lon,lat,neighbors,tot_dem,tot_gop,tot_votes,per_gop,median_income,median_income_black,median_income_native,median_income_asian,median_income_pacific,median_income_other,median_income_twoplus,median_income_hispanic,median_income_white,tot_edu_white,tot_edu_white_male,tot_edu_white_female,tot_edu_white_hsplus,tot_edu_white_male_hsplus,tot_edu_white_female_hsplus,tot_edu_white_4yplus,tot_edu_white_male_4yplus,tot_edu_white_female_4yplus,tot_edu_black,tot_edu_black_male,tot_edu_black_female,tot_edu_black_hsplus,tot_edu_black_male_hsplus,tot_edu_black_female_hsplus,tot_edu_black_4yplus,tot_edu_black_male_4yplus,tot_edu_black_female_4yplus,tot_edu_native,tot_edu_native_male,tot_edu_native_female,tot_edu_native_hsplus,tot_edu_native_male_hsplus,tot_edu_native_female_hsplus,tot_edu_native_4yplus,tot_edu_native_male_4yplus,tot_edu_native_female_4yplus,tot_edu_asian,tot_edu_asian_male,tot_edu_asian_female,tot_edu_asian_hsplus,tot_edu_asian_male_hsplus,tot_edu_asian_female_hsplus,tot_edu_asian_4yplus,tot_edu_asian_male_4yplus,tot_edu_asian_female_4yplus,tot_edu_pacific,tot_edu_pacific_male,tot_edu_pacific_female,tot_edu_pacific_hsplus,tot_edu_pacific_male_hsplus,tot_edu_pacific_female_hsplus,tot_edu_pacific_4yplus,tot_edu_pacific_male_4yplus,tot_edu_pacific_female_4yplus,tot_edu_other,tot_edu_other_male,tot_edu_other_female,tot_edu_other_hsplus,tot_edu_other_male_hsplus,tot_edu_other_female_hsplus,tot_edu_other_4yplus,tot_edu_other_male_4yplus,tot_edu_other_female_4yplus,tot_edu_twoplus,tot_edu_twoplus_male,tot_edu_twoplus_female,tot_edu_twoplus_hsplus,tot_edu_twoplus_male_hsplus,tot_edu_twoplus_female_hsplus,tot_edu_twoplus_4yplus,tot_edu_twoplus_male_4yplus,tot_edu_twoplus_female_4yplus,tot_edu_hispanic,tot_edu_hispanic_male,tot_edu_hispanic_female,tot_edu_hispanic_hsplus,tot_edu_hispanic_male_hsplus,tot_edu_hispanic_female_hsplus,tot_edu_hispanic_4yplus,tot_edu_hispanic_male_4yplus,tot_edu_hispanic_female_4yplus,pop_25p,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,edu,edu_white,per_edu_white_nohs,edu_white_male,per_edu_white_male_nohs,edu_white_female,per_edu_white_female_nohs,edu_black,per_edu_black_nohs,edu_black_male,per_edu_black_male_nohs,edu_black_female,per_edu_black_female_nohs,edu_native,per_edu_native_nohs,edu_native_male,per_edu_native_male_nohs,edu_native_female,per_edu_native_female_nohs,edu_asian,per_edu_asian_nohs,edu_asian_male,per_edu_asian_male_nohs,edu_asian_female,per_edu_asian_female_nohs,edu_pacific,per_edu_pacific_nohs,edu_pacific_male,per_edu_pacific_male_nohs,edu_pacific_female,per_edu_pacific_female_nohs,edu_other,per_edu_other_nohs,edu_other_male,per_edu_other_male_nohs,edu_other_female,per_edu_other_female_nohs,edu_twoplus,per_edu_twoplus_nohs,edu_twoplus_male,per_edu_twoplus_male_nohs,edu_twoplus_female,per_edu_twoplus_female_nohs,edu_hispanic,per_edu_hispanic_nohs,edu_hispanic_male,per_edu_hispanic_male_nohs,edu_hispanic_female,per_edu_hispanic_female_nohs,never,rarely,sometimes,frequently,always,mask,pop_density,per_male,per_female,per_pop_white_male,per_pop_white_female,per_pop_black_male,per_pop_black_fem

In [5]:
with open('../data/processed/geo_altair.p', 'rb') as f:
    geo_altair = pickle.load(f)

In [6]:
# #hide
# la = timezone('US/Pacific')
# last = nyt_df['date'].max()
# df_slice = nyt_df[nyt_df['date'] == last]
# df_slice.head()

In [ ]:
# #hide
# dem_df_ac = optimize(pd.read_csv('../data/processed/dem_df_ac.csv', 
#                                  dtype={'fips':'str', 'cluster':'int'}))
# print(dem_df_ac.shape)
# dem_df_ac.head()

# Exploring Correlations

In [ ]:
df = nyt_df.merge(info_df.loc[:, 'fips':'tot_pop'], on='fips', how='left')

In [8]:
def column_selector(info_df, columns=None, mask=[], exclude=[]):
    
    if columns is None:
        columns = []
        
    # only select from numeric columns
    all_columns = info_df.select_dtypes(include='number').columns.tolist()
        
    # if we want to look at a group of columns instead of inputting manually
    if len(mask+exclude) > 0:
        new_columns = all_columns
        if len(mask) > 0:
            new_columns = list(set([c for c in new_columns if all(m in set(re.findall('[a-z]+', c)) for m in mask)]))
        if len(exclude) > 0:
            new_columns = list(set([c for c in new_columns if all(e not in set(re.findall('[a-z]+', c)) for e in exclude)]))
        columns += new_columns
        
    return columns

In [52]:
def corr(x, y, w, useweight=True):
    x_ids = ~np.isnan(x)
    y_ids = ~np.isnan(y)
    ids = x_ids & y_ids
    if useweight:
        try:
            [xx, xy], [_, yy] = np.cov(x[ids], y[ids], aweights=w[ids])
        except:
            print(x.name)
            print(y.name)
    else:
        [xx, xy], [_, yy] = np.cov(x[ids], y[ids])
    c = xy / np.sqrt(xx * yy)
    return c

In [10]:
def df_merger(nyt_df, info_df, columns=None, features=None, date='latest', weight='tot_pop'):
    
    # make sure columns and features are valid
    all_features = nyt_df.select_dtypes(include='number').columns.tolist()
    features = [f for f in features if f in all_features]
    all_columns = info_df.select_dtypes(include='number').columns.tolist()
    columns = sorted([c for c in columns if c in all_columns])
    
    # merge nyt_df and info_df
    ## only process specific date and features
    left_columns = list(set(['date', 'county', 'state', 'fips'] + features))
    if date=='latest':
        left = nyt_df[nyt_df['date']==nyt_df['date'].max()][left_columns]
    elif date=='all':
        left = nyt_df[left_columns]
    else:
        left = nyt_df[nyt_df['date']==date][left_columns]
        
    ## only process specific columns of info_df
    right_columns = list(set(['fips', weight] + columns))
    right = info_df[right_columns]
    
    df = left.merge(right, on='fips', how='left')
    
    return df

In [23]:
def make_correlation_heatmap(
    nyt_df, info_df, columns=None, date='latest',
    features=[
        'cases_per_100k', 
        'new_cases_per_100k_15d',
        'delta_new_cases_per_100k_15d',
        'deaths_per_100k',
        'new_deaths_per_100k_15d',
        'delta_new_deaths_per_100k_15d',
        'mortality_rate',
        'mortality_rate_15d'
    ],
    useweight=True, weight='tot_pop', size=50, print_info=True
):
    
    df = df_merger(nyt_df, info_df, columns, features, date, weight)

    # build weighted correlation matrix from df
    x_columns = columns + features
    
    wcm = pd.DataFrame(index=columns, columns=x_columns)
    
    for y in columns:
        for x in x_columns:
            wcm.loc[y, x] = corr(df[y], df[x], df[weight])
    
    wcm = (wcm.reset_index().rename(columns={'index':'y_feature'}).dropna()
              .melt('y_feature', var_name='x_feature', value_name='corr'))
    wcm['corr'] = np.round(wcm['corr'].astype(float), 4)

    if print_info:
        print('positive correlations')
        print(
            wcm[(wcm['corr'] >= 0.4) & (wcm['corr'] != 1)]
            .sort_values(by=['corr', 'y_feature']).iloc[::2, :]
            .sort_values(by=['y_feature', 'x_feature'])
        )
        print('\nnegative correlations')
        print(
            wcm[(wcm['corr'] <= -0.4) & (wcm['corr'] != -1)]
            .sort_values(by=['corr', 'y_feature']).iloc[::2, :]
            .sort_values(by=['y_feature', 'x_feature'])
        )
    
    # build altair chart
    base = alt.Chart(wcm).encode(
        alt.X(
            'x_feature:O',
            sort=x_columns
        ),
        alt.Y(
            'y_feature:O',
            sort=columns
        )
    )
    heatmap = base.mark_rect().encode(
        color=alt.Color(
            'corr:Q',
            scale=alt.Scale(
                scheme='redblue',
                domain=[-1, 0, 1]
            )
        ),
        tooltip=[
            alt.Tooltip('x_feature:O'),
            alt.Tooltip('y_feature:O'),
            alt.Tooltip('corr:Q', title='correlation')
        ]
    )
    
    # text
    text = base.mark_text(baseline='middle').encode(
        text=alt.Text('corr:Q',format='.2f'),
        color=alt.condition(
            np.abs(alt.datum.corr) <= 0.5,
            alt.value('black'),
            alt.value('white')
        )
    )
    
    return (heatmap + text).configure_view(step=size)

In [49]:
nyt_df.groupby(by='date').get_group('2020-08-10').name

AttributeError: 'DataFrame' object has no attribute 'name'

In [24]:
columns=['per_gop', 'mask', 'edu', 'median_income', 'age_pop', 'pop_density']
columns += column_selector(info_df, mask=['per', 'pop'], exclude=['male', 'female', 'tot'])
columns = sorted(list(set(columns)))
make_correlation_heatmap(nyt_df, info_df, columns, features=[])

positive correlations
           y_feature         x_feature    corr
40               edu     median_income  0.7343
66               edu     per_pop_asian  0.4254
41              mask     median_income  0.4295
67              mask     per_pop_asian  0.4586
93              mask  per_pop_hispanic  0.4599
68     median_income     per_pop_asian  0.6061
147          per_gop     per_pop_white  0.6785
135    per_pop_asian   per_pop_twoplus  0.4415
139  per_pop_pacific   per_pop_twoplus  0.8468

negative correlations
            y_feature      x_feature    corr
53                edu        per_gop -0.4201
54               mask        per_gop -0.6458
145              mask  per_pop_white -0.5242
69            per_gop  per_pop_asian -0.5732
160           per_gop    pop_density -0.4718
148     per_pop_asian  per_pop_white -0.4925
149     per_pop_black  per_pop_white -0.4285
150  per_pop_hispanic  per_pop_white -0.7649


alt.LayerChart(...)

First, let's discuss features that not quite independent from each other:

selected positive correlations (> 0.4):
- educational attainment and median income
- educational attainment and percent asian
- mask discipline and median income
- mask discipline and percent asian
- mask discipline and percent hispanic
- median income and percent asian
- percent GOP and percent white

selected negative correlations (< -0.4):
- educational attainment and percent GOP
- mask discipline and percent GOP
- mask discipline and percent white
- percent Asian and percent GOP
- population density and percent GOP

It's important to understand these correlations, since they introduce possible confounding variables when exploring correlations with COVID-specific features (eg `new_cases`).

# Exploring Time Series Correlations

New COVID-19 cases/deaths have varied widely depending on time of year. While we could explore correlations with total cases and deaths, it would be more interesting to see how different factors are correlated depending on the date. We compare these features to a 15-day window of new cases and deaths.

In [61]:
def make_correlation_timeseries(
    nyt_df, info_df, columns=None, feature='new_cases_per_100k_15d', 
    useweight=True, weight='tot_pop'
):
    


1.5677231595552148

In [90]:
def make_correlation_timeseries(
    nyt_df, info_df, columns=None, feature='new_cases_per_100k_15d', 
    useweight=True, weight='tot_pop', divisions=1, divide_on=None
):
    
    def _timeseries(left, right):
        
        df = df_merger(
            left, right, columns=columns, features=[feature], date='all', 
            weight=weight
        ).dropna(axis=0)

        # set up skeleton for weighted correlation matrix
        wcm = pd.DataFrame(index=sorted(df['date'].unique()), columns=columns)

        # slice out the dataframe so we process less data

        grouped = df.groupby(by='date')

        # populate weighted correlation matrix
        # FIND A WAY TO OPTIMIZE THIS CODE
        for d, g in grouped:
            if len(g) > 1:
                for c in columns:
                    wcm.loc[d, c] = corr(g[c], g[feature], g[weight], useweight=useweight)

        width = 5*len(wcm.dropna())
        # convert to long-form dataframe for altair
        wcm = (wcm.reset_index().rename(columns={'index':'date'}).dropna()
                  .melt('date', var_name='feature', value_name='corr'))
        wcm['corr'] = wcm['corr'].astype(float)

        # build altair chart
        base = alt.Chart(wcm).encode(
            alt.X(
                'monthdate(date):T',
                axis=alt.Axis(format='%b %d')
            ),
            alt.Y('feature:O')
        )
        heatmap = base.mark_rect().encode(
            color=alt.Color(
                'corr:Q',
                scale=alt.Scale(
                    scheme='redblue',
                    domain=[-0.5, 0, 0.5]
                )
            ),
            tooltip=[
                alt.Tooltip('date:T', title='date'),
                alt.Tooltip('feature:O'),
                alt.Tooltip('corr:Q', title='correlation')
            ]
        )

        return heatmap
    
    if divisions==1:
        return _timeseries(nyt_df, info_df).configure_view(step=15, continuousWidth=750)
    
    elif (divisions>1) and (divide_on is not None):
#         qcut_labels = np.arange(divisions)
        info_df['qcut'] = pd.qcut(info_df[divide_on], divisions)
        charts = []
        for i in sorted(info_df['qcut'].unique()):
            charts.append(
                _timeseries(nyt_df, info_df[info_df['qcut']==i]).properties(
                    title=f'{divide_on} = {str(i)}'
                )
            )
        return alt.vconcat(*charts[::-1]).configure_view(
            step=15, continuousWidth=750
        ).resolve_scale(x='shared')

In [76]:
make_correlation_timeseries(
    nyt_df, 
    info_df, 
    columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], 
    feature='new_cases_per_100k_15d'
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.Chart(...)

In [91]:
make_correlation_timeseries(
    nyt_df, 
    info_df, 
    columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], 
    feature='new_cases_per_100k_15d',
    divisions=5,
    divide_on='pop_density'
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.VConcatChart(...)

## population density, percent GOP, mask, edu, age

Given that we are discussing the rate in which a disease is spread, it would make sense that population density (`pop_density`) has the highest positive correlation with number of total cases per capita (`cases_per_100k`).

Other features are not completely independent from population density. 
- percent GOP voters is negatively correlated with population density
- mask discipline, educational attainment, and average age is weakly correlated with population density  

Because of this, it makes sense that the percentage of GOP voters is negatively correlated with total cases per capita. However, educational attainment and average age are still negatively correlated with total cases per capita, which may indicate a stronger relationship between these features and total cases.

It would most likely be more useful to look at *recent* cases per capita (`new_cases_per_100k_15d`), as more populous areas will have higher case totals overall--so much so that their earlier numbers will likely still eclipse recent statistics of less populous counties (New York City was hit especially hard in mid-March).

Notice how in recent windows, population density, mask discipline, and educational attainment all have *lower* correlations, whereas percent of GOP voters have an increase in correlation.

In [25]:
make_correlation_heatmap(nyt_df, info_df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'])

positive correlations
      y_feature       x_feature    corr
55  pop_density  mortality_rate  0.4827

negative correlations
   y_feature        x_feature    corr
8        edu          per_gop -0.4201
7       mask          per_gop -0.6458
41   per_gop  deaths_per_100k -0.4035
1    per_gop      pop_density -0.4718


alt.LayerChart(...)

In [66]:
make_correlation_timeseries(nyt_df, info_df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], feature='new_cases_per_100k_15d')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.Chart(...)

In [64]:
alt.vconcat(
    make_correlation_timeseries(nyt_df, info_df[info_df['pop_density'] < info_df['pop_density'].median()], columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop']),
    make_correlation_timeseries(nyt_df, info_df[info_df['pop_density'] >= info_df['pop_density'].median()], columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'])
).resolve_scale(x='shared')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.VConcatChart(...)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.Chart(...)

In [16]:
make_correlation_timeseries(nyt_df, info_df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], feature='new_deaths_per_100k_15d')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [22]:
make_correlation_timeseries(nyt_df, info_df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], feature='mortality_rate_15d', useweight=False)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Degrees of freedom <= 0 for slice
  
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

Even with NYC out of the picture, we see that COVID-19 primarily hit urban centers first and has since moved on to more rural areas.

In [26]:
make_correlation_timeseries(nyt_df[nyt_df['fips']!='36NYC'], info_df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

## age

In [28]:
columns = column_selector(info_df, mask=['age'], exclude=['tot', 'male', 'female'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [29]:
make_correlation_heatmap(nyt_df, info_df, columns, print_info=False)

alt.LayerChart(...)

## edu

In general, counties with lower educational attainment seem to be experiencing a higher density of cases than those with higher educational attainment. This is true across all ethnicities, but what's interesting is how hispanic populations are disproportionately affected.

In [30]:
columns = column_selector(info_df, mask=['edu'], exclude=['tot', 'male', 'female'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

## ethnic percentages

In [32]:
columns = column_selector(info_df, mask=['per', 'pop'], exclude=['tot', 'male', 'female'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

## white age, education, income

In [33]:
columns = column_selector(info_df, mask=['white'], exclude=['tot'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [35]:
columns = ['per_pop_white', 'pop_density', 'per_gop', 'mask', 'edu', 'age_pop', 'median_income']
make_correlation_heatmap(nyt_df, info_df, columns, print_info=False)

alt.LayerChart(...)

## hispanic age, education, income

In [92]:
make_correlation_timeseries(
    nyt_df, 
    info_df, 
    columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], 
    feature='new_cases_per_100k_15d',
    divisions=5,
    divide_on='per_pop_hispanic'
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


alt.VConcatChart(...)

In [37]:
columns = column_selector(info_df, mask=['hispanic'], exclude=['tot'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [38]:
columns = ['per_pop_hispanic', 'pop_density', 'per_gop', 'mask', 'edu_hispanic', 'age_pop_hispanic', 'median_income_hispanic']
make_correlation_heatmap(nyt_df, info_df, columns, print_info=False)

alt.LayerChart(...)

## black age, education, income

In [39]:
columns = column_selector(info_df, mask=['black'], exclude=['tot'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

## asian age, education, income

In [40]:
columns = column_selector(info_df, mask=['asian'], exclude=['tot'])
make_correlation_timeseries(nyt_df, info_df, columns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [240]:
make_correlation_timeseries(df, columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], feature='new_deaths_per_100k_15d')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

In [241]:
make_correlation_timeseries(
    df[df['fips']!='36NYC'], 
    columns=['pop_density', 'per_gop', 'mask', 'edu', 'age_pop'], 
    feature='new_deaths_per_100k_15d'
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


alt.Chart(...)

The following are, upon inspection, positively correlated with the total number of cases and deaths per cluster:
- population density (`pop_density`)
- percent black (`per_black`)
- percent hispanic (`per_hispanic`)
- mask discipline (`mask`)

The following are negatively correlated with the total number of cases and deaths per county:
- percent white (`per_white`)
- adjusted percent votes GOP in the 2016 general election (`per_gop`)

## new cases and deaths

The following features are positively correlated with the number of *new* cases and deaths per cluster:
- percent black
- percent hispanic

And the following are negatively correlated:
- percent white
- percent of population who voted in the general election 2016 (`per_total_votes`)
- median income (`median_income`)
- education coefficient (`education`)

# Scatter Plots (with color and size axes)

In each chart:

- size -- total cases

In [ ]:
#hide
#https://colorbrewer2.org/

color_dict = dict(
    # blue-red
    per_gop=dict(
        range=['#2166ac', '#92c5de', '#F7F7F7', '#f4a582', '#b2182b'],
        mid=0.5
    ),
    # pink-green
    mask=dict(
        range=['#c51b7d', '#f1b6da', '#F7F7F7', '#b8e186', '#4d9221'],
        mid=info_df['mask'].median()
    ),
    age_pop=dict(
        range=['#c51b7d', '#f1b6da', '#F7F7F7', '#b8e186', '#4d9221'],
        mid=info_df['age_pop'].median()
    ),
    # purple-orange
    median_income=dict(
        range=['#542788', '#b2abd2', '#F7F7F7', '#fdb863', '#b35806'],
        mid=61937
    ),
    # green-purple
    education=dict(
        range=['#1b7837', '#a6dba0', '#F7F7F7', '#c2a5cf', '#762a83'],
        mid=info_df['edu'].median()
    )
)

# COVID-19 Map

In [ ]:
with open('../data/processed/geo_altair.p', 'rb') as f:
    geo_altair = pickle.load(f)

In [ ]:
def make_map(df, c_col, s_col='cases', 
             c_range=None, c_mid=None):
    
#     base_cols = ['cases', 'cases_per_100k', 'deaths', 'deaths_per_100k', 'mortality_rate']
#     if s_col not in base_cols:
#         s_col = 'cases'

#     cols = ['date', 'state', 'county', 'total_pop', 'lat', 'lon', 'fips'] + base_cols + [c_col, s_col]
#     cols = list(set(cols))
#     smax = np.max(df[s_col])
#     smax = smax + (-smax) % 10

    if c_range==None:
        c_range = color_dict[c_col]['range']
    if c_mid==None:
        c_mid = color_dict[c_col]['mid']

#     df = df[cols]

    states = alt.topo_feature(data.us_10m.url, feature='states')
    

    background = alt.Chart(states).mark_geoshape(
        fill='#F2F2F2',
        stroke='white'
    ).properties(
        title='total cases',
        width=720,
        height=480
    ).project('albersUsa')
    
    counties = alt.Chart(geo_altair).mark_geoshape(
        stroke='white'
    ).encode(
        color=alt.Color(
            f'{c_col}:Q',
            scale=alt.Scale(
                range=c_range,
                domain=[np.percentile(df[c_col], 1),
                        c_mid,
                        np.percentile(df[c_col], 99)],
                interpolate={
                    'type':'rgb', 
                    'gamma':0.75
                }
            ),
        ),
        tooltip=[
            'state:N', 'county:N', 'cases:Q', 'deaths:Q', 
            alt.Tooltip(
                'mortality_rate:Q',
                format='.3f'
            ),
            alt.Tooltip(
                f'{s_col}:Q',
                format='.2f',
            ),
            alt.Tooltip(
                f'{c_col}:Q',
                format='.2f',
            )
        ]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=df[cols], key='fips', fields=cols)
    )
    
    return background + counties

In [ ]:
make_map(df_slice, c_col='new_cases_per_100k_15sg',
         c_range=['#2c7bb6','#abd9e9','#ffffbf','#fdae61','#d7191c'],
         c_mid=0)

In [ ]:
def make_map(df, c_col, s_col='cases', 
             c_range=None, c_mid=None):
    
    base_cols = ['cases', 'cases_per_100k', 'deaths', 'deaths_per_100k', 'mortality_rate']
#     if s_col not in base_cols:
#         s_col = 'cases'

    cols = ['date', 'state', 'county'] + base_cols
    cols = list(set(cols))
    smax = np.max(df[s_col])
    smax = smax + (-smax) % 10

    if c_range==None:
        c_range = color_dict[c_col]['range']
    if c_mid==None:
        c_mid = color_dict[c_col]['mid']

    df = df[cols]

    states = alt.topo_feature(data.us_10m.url, feature='states')
    

    background = alt.Chart(states).mark_geoshape(
        fill='#F2F2F2',
        stroke='white'
    ).properties(
        title='total cases',
        width=720,
        height=480
    ).project('albersUsa')
    
    counties = alt.Chart(geo_altair).mark_geoshape(
        stroke='white'
    ).encode(
        color=alt.Color(
            f'{c_col}:Q',
            scale=alt.Scale(
                range=c_range,
                domain=[np.percentile(df[c_col], 1),
                        c_mid,
                        np.percentile(df[c_col], 99)],
                interpolate={
                    'type':'rgb', 
                    'gamma':0.75
                }
            ),
        ),
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=df[cols], key='fips', fields=cols)
    )

    counties = alt.Chart(df).mark_circle(
        stroke='black', strokeWidth=0.25, opacity=0.8
    ).encode(
        latitude='lat:Q',
        longitude='lon:Q',
        size=alt.Size(
            f'{s_col}:Q',
#             scale=alt.Scale(
#                 domain=[1,smax],
#                 range=[10,2000]
#             ),
        ),
        
        tooltip=[
            'state:N', 'county:N', 'cases:Q', 'deaths:Q', 
            alt.Tooltip(
                'mortality_rate:Q',
                format='.3f'
            ),
            alt.Tooltip(
                f'{s_col}:Q',
                format='.2f',
            ),
            alt.Tooltip(
                f'{c_col}:Q',
                format='.2f',
            )
        ]
    )
    
    return background + circles

In [ ]:
make_map(df_slice, c_col='delta_new_cases_per_100k_15sg', s_col='new_cases_per_100k_15sg',
         c_range=['#2c7bb6','#abd9e9','#ffffbf','#fdae61','#d7191c'],
         c_mid=0)

## COVID-19 Density and Mortality Rate vs. Population Density

In [ ]:
df_slice.merge(info_df[['fips', 'tot_pop', 'pop_density', 'per_gop']], on='fips', how='left').info()

In [ ]:
def make_bubble_chart(df, x_col, y_col, c_col, s_col='cases',
                      x_scale='linear', y_scale='linear', 
                      c_range=None, c_mid=None, line=True, split=False, clip=False):
    
#     base_cols = ['cases', 'cases_per_100k', 'deaths', 'deaths_per_100k', 'mortality_rate']
#     if s_col not in base_cols:
#         s_col = 'cases'

#     cols = ['date', 'state', 'county', 'total_pop'] + base_cols + [x_col, y_col, c_col]
#     cols = list(set(cols))
    smax = np.max(df[s_col])
    smax = smax + (-smax) % 10

    if c_range==None:
        c_range = color_dict[c_col]['range']
    if c_mid==None:
        c_mid = color_dict[c_col]['mid']

#     df = df[cols]
    
    base = alt.Chart(df).properties(
        title=f'{y_col} vs. {x_col} ({c_col}, cases)',
        width=720,
        height=480
    )
    
    circles = base.mark_circle(stroke='black', strokeWidth=0.25, opacity=0.8).encode(
        x=alt.X(
            f'{x_col}:Q',
            scale=alt.Scale(type=x_scale, zero=False),
            impute=alt.ImputeParams(value=0)
        ),
        y=alt.Y(
            f'{y_col}:Q',
            scale=alt.Scale(type=y_scale, zero=False)
        ),
        size=alt.Size(
            f'{s_col}:Q',
            scale=alt.Scale(
                domain=[1,smax],
                range=[10,2000]
            ),
        ),
        color=alt.Color(
            f'{c_col}:Q',
            scale=alt.Scale(
                range=c_range,
                domain=[np.percentile(info_df[c_col], 1),
                        c_mid,
                        np.percentile(info_df[c_col], 99)],
                interpolate={
                    'type':'rgb', 
                    'gamma':0.75
                }
            ),
        ),
        tooltip=[
            'state:N', 'county:N', 'cases:Q', 'deaths:Q', 
            alt.Tooltip(
                'mortality_rate:Q',
                format='.3f'
            ),
            alt.Tooltip(
                f'{c_col}:Q',
                format='.2f',
            ),
            alt.Tooltip(
                f'{y_col}:Q',
                format='.2f',
            ),
            alt.Tooltip(
                f'{x_col}:Q',
                format='.2f',
            )
        ]
    ).transform_lookup(
        lookup='fips',
        from_=alt.LookupData(data=info_df, key='fips', fields=[x_col, c_col])
    )

    def make_line(df, split=False, side=None):
        df_ = df.merge(info_df[['fips', 'tot_pop', x_col, c_col]], on='fips', how='left')
        if split:
            if side=='gt':
                df_ = df_[df_[c_col]>c_mid]
                color = c_range[-1]
            elif side=='lt':
                df_ = df_[df_[c_col]<=c_mid]
                color = c_range[0]
        else:
            color = 'black'
        pfit = poly.polyfit(
            df_[x_col], 
            df_[y_col], 
            1, 
            w=df_['tot_pop']
        )
        fit = poly.polyval(np.unique(df_[x_col]), pfit)
        line_df = pd.DataFrame({
            'x': np.unique(df_[x_col]),
            'y': fit
        })
        return alt.Chart(line_df).mark_line(
            color=color,
            strokeDash=[2,1],
            clip=True
        ).encode(
            x='x:Q',
            y=alt.Y(
                'y:Q', 
                scale=alt.Scale(domain=[
                    df_[y_col].min(),
                    df_[y_col].max()
                ])
            )
        )
    
    if line:
        if split:
            _line0 = make_line(df, True, 'lt')
            _line1 = make_line(df, True, 'gt')
            return circles + _line0 + _line1
        else:
            _line = make_line(df)
            return circles + _line
        
    else:
        return circles

In [ ]:
make_bubble_chart(df_slice, 'pop_density', 'cases_per_100k', 'age_pop',
                  x_scale='sqrt', y_scale='sqrt', split=False)

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'cases_per_100k', 'per_gop',
                  x_scale='linear', y_scale='sqrt', split=False)

By far the largest predictor of total cases/deaths is population density. This does not appear to be an unfair characterization, given the fact that the disease spreads from person-to-person. Dense urban centers tend to lean Democratic, whereas sparse rural counties tend to lean Republican. Therefore, it makes sense that urban Democratic centers are being hit harder than their rural Republican counterparts.

The effects of urban density are further illustrated when discussing the mortality rate. In New York City, over 10% of those diagnosed have passed due to complications from the virus.

In [ ]:
make_bubble_chart(df_slice, 'pop_density', 'mortality_rate', 'per_gop',
                  x_scale='sqrt', y_scale='sqrt', line=True, split=True)

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'mortality_rate', 'per_gop',
                  x_scale='linear', y_scale='sqrt', line=True, split=False)

## COVID-19 Density vs. Mask Discipline

In [ ]:
make_bubble_chart(df_slice, 'mask', 'cases_per_100k', 'per_gop',
                  x_scale='sqrt', y_scale='sqrt', line=True, split=False)

## COVID-19 Density vs. Educational Attainment

Educational attainment (`edu`) is defined as the weighted average of the highest degree of education for persons 25 and older within a given county:

- no HS -- 0
- some HS -- 1
- HS diploma or GED -- 2
- some college -- 3
- associate's degree -- 4
- bachelor's degree -- 5
- graduate or professional degree - 6

In [ ]:
make_bubble_chart(df_slice, 'education', 'cases_per_100k', 'per_gop',
                  x_scale='sqrt', y_scale='sqrt', line=True, split=False)

In [ ]:
#hide_input
params = dict(
    chart_title='Cases per 100k vs. Educational Attainment (% GOP)',
    x_col='education',
    x_scale='linear',
    y_col='cases_per_100k',
    y_scale='sqrt',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

case_edu = make_bubble_chart(df_slice, params)
case_edu

In [ ]:
#hide_input
params = dict(
    chart_title='Mortality Rate vs. Educational Attainment (% GOP)',
    x_col='edu',
    x_scale='linear',
    y_col='new_cases_per_100k_15sg',
    y_scale='sqrt',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

new_case_edu = make_bubble_chart(df_slice, params)
new_case_edu

## COVID-19 Density vs. Mask Discipline

Mask discipline (`edu`) is defined as the weighted average of the answer to the following question: "How often do you wear a mask in public when you expect to be within six feet of another person?"

- never -- 0
- rarely -- 1
- sometimes -- 2
- frequently -- 3
- always -- 4

In [ ]:
#hide_input
params = dict(
    chart_title='Cases per 100k vs. Mask Discipline (% GOP)',
    x_col='mask',
    x_scale='linear',
    y_col='cases_per_100k',
    y_scale='sqrt',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

mask_edu = make_bubble_chart(df_slice, params)
mask_edu

In [ ]:
#hide_input
params = dict(
    chart_title='New Cases per 100k vs. Mask Discipline (% GOP)',
    x_col='mask',
    x_scale='linear',
    y_col='new_cases_per_100k_15sg',
    y_scale='sqrt',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

mask_edu = make_bubble_chart(df_slice, params)
mask_edu

In [ ]:
#hide_input
params = dict(
    chart_title='Change in New Cases per 100k vs. Mask Discipline (% GOP)',
    x_col='mask',
    x_scale='linear',
    y_col='delta_new_cases_per_100k_15sg',
    y_scale='sqrt',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

mask_edu = make_bubble_chart(df_slice, params)
mask_edu

In [ ]:
#hide_input
params = dict(
    chart_title='Case Density vs. Population Density (Mask Discipline)',
    x_col='pop_density',
    x_scale='sqrt',
    y_col='new_cases_per_100k_15sg',
    y_scale='sqrt',
    s_col='cases',
    c_col='mask',
    w_col='total_pop'
)

mask_edu = make_bubble_chart(df_slice, params)
mask_edu

# New Cases per 100k vs. Population Density, Education, and Mask Discipline

Mask use seems to increase with educational attainment and is somehow a partisan issue, with Republican counties having worse mask discipline. With the size of the circles representing new cases, we see that counties with a high education attainment and mask discipline ratines (top-right quadrant) seem to have a lower number of new cases than those with low education and mask discipline (bottom-left quadrant).

In [ ]:
#hide_input
params = dict(
    chart_title='Case Density vs. Population Density, Education, and Mask Discipline',
    x_col='edu',
    x_scale='linear',
    y_col='mask',
    y_scale='linear',
    s_col='new_cases_per_100k_15sg',
    c_col='per_gop',
    w_col='total_pop'
)

medley = make_bubble_chart(df_slice, params)
medley

This is not particularly easy to read, so maybe we'll use matplotlib instead.

In [ ]:
%matplotlib inline

sns.set_style('whitegrid')

fig = plt.figure(figsize=(17,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_slice['edu'], 
           df_slice['mask'], 
           df_slice['new_cases_per_100k_15sg'], 
           s=df_slice['cases'] / 200,
           c=df_slice['per_gop'],
           cmap='RdBu_r')
ax.set_xlabel('educational attainment')
ax.set_ylabel('mask discipline')
ax.set_zlabel('new cases per 100k')
ax.view_init(20,280)
plt.show()

## New Cases per 100k vs. Percent GOP

Recent COVID-19 cases, however, seem to be more prevalent in more GOP-weighted counties.

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'new_cases_per_100k_15sg', 'per_gop',
                  x_scale='linear', y_scale='sqrt', line=True, split=False)

This is most likely skewed by the fact that New York City -- a prior epicenter of COVID-19 cases and the largest group in this dataset -- has relatively few cases now.

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'new_cases_per_100k_15sg', 'per_gop',
                  x_scale='linear', y_scale='sqrt', line=True, split=True)

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'delta_new_cases_per_100k_15sg', 'per_gop',
                  x_scale='linear', y_scale='sqrt', line=True, split=False)

In [ ]:
make_bubble_chart(df_slice, 'per_gop', 'delta_new_cases_per_100k_15sg', 'per_gop',
                  x_scale='linear', y_scale='sqrt', line=True, split=True)

## Change in New Cases per 100k vs. Percent GOP

Changes in cases do not seem to be predicted by political affiliation.

In [ ]:
#hide_input
params = dict(
    chart_title='Change in New Cases vs. Percent GOP',
    x_col='per_gop',
    x_scale='linear',
    y_col='delta_new_cases_per_100k_15sg',
    y_scale='linear',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

del_case_gop = make_bubble_chart(df_slice, params)
del_case_gop

In [ ]:
#hide_input
params = dict(
    chart_title='Change in New Cases vs. Percent GOP',
    x_col='per_gop',
    x_scale='linear',
    y_col='delta_new_cases_per_100k_15sg',
    y_scale='linear',
    s_col='cases',
    c_col='per_gop',
    w_col='total_pop'
)

del_case_gop_split = make_bubble_chart(df_slice, params, split=True)
del_case_gop_split

## Case Rate Charts

Counties in the upper right portion of this chart face the highest risk of a worsening pandemic.

In [ ]:
make_bubble_chart(df_slice, 'new_cases_per_100k_15sg', 'delta_new_cases_per_100k_15sg', 'per_gop',
                  x_scale='sqrt', y_scale='sqrt', line=True, split=True)